In [2]:
using Distributions
using ReactiveMP
using Rx

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1273
┌ Warning: Package ReactiveMP does not have Rx in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rx as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rx into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [36]:
mutable struct InferenceActor <: Actor{AbstractMessage}
    index :: Int
    size  :: Int
    data  :: Vector{Float64}
    means :: Vector{Float64}
    
    y          :: ObservedVariable
    e_mean     :: EstimatedVariable
    e_variance :: EstimatedVariable
    
    InferenceActor(data::Vector{Float64}, y::ObservedVariable, e_mean::EstimatedVariable, e_variance::EstimatedVariable) = begin
        size  = length(data)
        means = Vector{Float64}(undef, size)
        return new(1, size, data, means, y, e_mean, e_variance)
    end
end

function update!(actor::InferenceActor, mean::Float64, variance::Float64)
    next!(actor.y.values, actor.data[actor.index])
    next!(actor.e_mean.values, mean)
    next!(actor.e_variance.values, variance)
end

function stop!(actor::InferenceActor)
    complete!(actor.y.values)
    complete!(actor.e_mean.values)
    complete!(actor.e_variance.values)
end

function Rx.on_next!(actor::InferenceActor, data::AbstractMessage)
    m = mean(data.distribution)
    v = var(data.distribution)
    
    actor.means[actor.index] = m
    
    actor.index += 1
    
    @async begin
        if actor.index < actor.size 
            update!(actor, m, v)
        else
            stop!(actor)
        end
    end
end

Rx.on_error!(actor::InferenceActor, err) = error(err)
Rx.on_complete!(actor::InferenceActor) = println("Completed: $(time())")

Base.show(io::IO, actor::InferenceActor) = print(io, "InferenceActor")

In [37]:
xPrevAddConstantNode = AdditionNode("xPrevAddConstantNode");
addConstant = ConstantVariable("addConstant", 1.0, xPrevAddConstantNode.in2);

estimatingXNode    = GaussianMeanVarianceFactorNode("estimatingXWithGaussianNode");
estimatedXMean     = EstimatedVariable("estimatedXMean", estimatingXNode.mean);
estimatedXVariance = EstimatedVariable("estimatedXVariance", estimatingXNode.variance);

xPrev = RandomVariable("xPrev", estimatingXNode.value, xPrevAddConstantNode.in1);

noiseNode     = GaussianMeanVarianceFactorNode("noiseNode");
noiseMean     = ConstantVariable("noiseMean", 0.0, noiseNode.mean);
noiseVariance = ConstantVariable("noiseVariance", 1.0, noiseNode.variance);

addXAndNoiseNode = AdditionNode("addXAndNoiseNode");

x = RandomVariable("x", xPrevAddConstantNode.out, addXAndNoiseNode.in1);
n = RandomVariable("n", noiseNode.value, addXAndNoiseNode.in2);
y = ObservedVariable("y", addXAndNoiseNode.out);

In [38]:
n = 10000
data = [ i for i::Float64 in 1:n ] + rand(Normal(0.0, 1.0), n);

In [39]:
actor = InferenceActor(data, y, estimatedXMean, estimatedXVariance);

In [40]:
function try_reactive(actor::InferenceActor, x)
    println(time())
    subscribe!(inference(x), actor)
    update!(actor, 0.0, 1000.0)
    return actor
end

try_reactive (generic function with 1 method)

In [41]:
@time try_reactive(actor, x);

1.577126554300865e9
  0.107737 seconds (260.71 k allocations: 12.975 MiB)
Completed: 1.577126555687583e9


In [42]:
1.577126555687583e9 - 1.577126554300865e9

1.3867180347442627

In [187]:
1.577124479150774e9 - 1.577124477606584e9 

1.5441899299621582